<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%2313.%EC%9C%A0%EB%8B%88%EC%BD%94%EB%93%9C%EB%AC%B8%EC%9E%90%EC%97%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

    유니코드 : 표준 인코딩시스템

    텐서플로에서 유니코드 문자열을 표현 -> 유니코드 문자열 조작 

In [2]:
import tensorflow as tf

In [3]:
# 텐서플로의 기본 tf.string dtype은 바이트 문자열, 유니코드 문자열은 기본적으로 utf-8로 인코딩
tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

In [5]:
tf.constant([u'You re', u'welcome']).shape

TensorShape([2])

In [6]:
# 유니코드 표현

# 1. string 스칼라 - 코드 포인트의 시퀀스가 알려진 문자 인코딩을 사용해 인코딩
# 2. int32 벡터 - 위치마다 개별 코드 포인터를 포함

In [12]:
# ex) 语言处理

# 1. UTF-8로 인코딩된 string 스칼라로 표현한 유니코드 문자열

text_utf8 = tf.constant(u'语言处理')
print('[#1]','\n',text_utf8)

#2. UTF-16-BE로 인코딩된 string 스칼라로 표현한 유니코드 문자열
text_utf16be = tf.constant(u'语言处理'.encode('UTF-16-BE'))
print('[#2]','\n',text_utf16be)

#3. 유니코드 코드 포인트의 벡터로 표현한 유니코드 문자열
text_chars = tf.constant([ord(char) for char in u"语言处理"])
print('[#3]','\n',text_chars)

[#1] 
 tf.Tensor(b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86', shape=(), dtype=string)
[#2] 
 tf.Tensor(b'\x8b\xed\x8a\x00Y\x04t\x06', shape=(), dtype=string)
[#3] 
 tf.Tensor([35821 35328 22788 29702], shape=(4,), dtype=int32)


In [13]:
# 표현간 변환

tf.strings.unicode_decode(text_utf8, input_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 22788, 29702], dtype=int32)>

In [14]:
# 다른 인코딩으로 변환
tf.strings.unicode_transcode(text_utf8,
                             input_encoding = 'UTF-8',
                             output_encoding = 'UTF-16-BE')

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Y\x04t\x06'>

In [16]:
# 배치 차원
# 여러 개의 문자열을 디코딩 시 문자열마다 포함된 문자의 개수는 동일하지 않음.
# 반환되는 값은 tf.RaggedTensor로 가장 안쪽 차원의 크기가 문자열에 포함된 문자의 개수에 따라 결정

batch_utf8 = [s.encode('UTF-8') for s in [u'hÃllo',  u'What is the weather tomorrow',  u'Göödnight', u'😊']]
batch_utf8

[b'h\xc3\x83llo',
 b'What is the weather tomorrow',
 b'G\xc3\xb6\xc3\xb6dnight',
 b'\xf0\x9f\x98\x8a']

In [20]:
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding = 'UTF-8')
batch_chars_ragged.to_list()[:2]

[[104, 195, 108, 108, 111],
 [87,
  104,
  97,
  116,
  32,
  105,
  115,
  32,
  116,
  104,
  101,
  32,
  119,
  101,
  97,
  116,
  104,
  101,
  114,
  32,
  116,
  111,
  109,
  111,
  114,
  114,
  111,
  119]]

In [22]:
for sentence_chars in batch_chars_ragged.to_list() :
    print(sentence_chars)

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


In [24]:
# tf.RaggedTensor 바로 사용 or 패딩사용하여 tf.Tensor로 변환
# or tf.RaggedTensor.to_tensor와 tf.RaggedTensor.to_sparse 메서드를 사용해 tf.SparseTensor로 변환 가능

batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]]


In [27]:
batch_chars_sparse = batch_chars_ragged.to_sparse()

In [28]:
# 길이가 같은 여러 문자열을 인코딩 시 tf.Tensor입력으로 사용

tf.strings.unicode_encode([
                           [99 , 97 , 116],
                           [100 , 111 , 103],
                           [99 , 111 , 119]
], output_encoding='UTF-8')

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

In [29]:
# 길이가 다른 여러 문자열을 인코딩 시 tf.RaggedTensor를 입력으로 사용

tf.strings.unicode_encode(batch_chars_ragged,
                          output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [30]:
# 패딩된 텐서나 희소(Sparse) 텐서는 unicode_encode를 호출하기 전 tf.RaggedTensor로 바꿈

# 희소 텐서
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [31]:
# 패딩 텐서
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=1),
    output_encoding = 'UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd',
       b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd',
       b'\xf0\x9f\x98\x8a\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd\xef\xbf\xbd'],

In [36]:
# 유니코드 연산
# [길이]
# tf.strings.length  : 계산해야 할 길이를 나타내는 unit인자를 요구 
# 기본 단위 : byte, string에 포함된 유니코드 코드 포인트의 수를 파악하기 위해 'UTF8_CHAR' Or 'UTF16_CHAR 설정가능

# UTF8에서 마지막 문자는 4바이트를 차지합니다.
thanks = u'Thanks 😊'.encode('UTF-8')

num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} 바이트; {}개의 UTF-8 문자'.format(num_bytes, num_chars))

11 바이트; 8개의 UTF-8 문자


In [38]:
# 부분문자열
# tf.strings.substr : unit 매개 변수 값을 사용해 pos, len 매개 변수로 지정된 문자열 종류 결정
tf.strings.substr(thanks, pos=7, len=1).numpy()

b'\xf0'

In [39]:
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

b'\xf0\x9f\x98\x8a'
